## Descarga y formatting del CSV de los datos

In [1]:
!wget -O "airbnb-listings.csv" "https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B"

--2023-07-16 15:16:48--  https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B
Resolving public.opendatasoft.com (public.opendatasoft.com)... 34.248.20.69, 34.249.199.226
Connecting to public.opendatasoft.com (public.opendatasoft.com)|34.248.20.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘airbnb-listings.csv’

airbnb-listings.csv     [        <=>         ]  54.31M  2.96MB/s    in 18s     

2023-07-16 15:17:07 (3.00 MB/s) - ‘airbnb-listings.csv’ saved [56950929]



In [2]:
import cv2
import numpy
import pandas
import imageio.v3 as io

from tqdm import tqdm
from typing import Optional, Union
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


In [3]:
full_df = pandas.read_csv("airbnb-listings.csv", sep = ';')


# División en train y test
train, test = train_test_split(full_df, test_size=0.2, shuffle=True, random_state=0)

# División del conjunto de prueba en test y validation
test, validation = train_test_split(test, test_size=0.5, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de entrenamiento: {train.shape}')
print(f'Dimensiones del dataset de prueba: {test.shape}')
print(f'Dimensiones del dataset de validación: {validation.shape}')

# Guardar los conjuntos de datos
train.to_csv('train.csv', sep=';', decimal='.', index=False)
test.to_csv('test.csv', sep=';', decimal='.', index=False)
validation.to_csv('validation.csv', sep=';', decimal='.', index=False)


Dimensiones del dataset de entrenamiento: (11200, 89)
Dimensiones del dataset de prueba: (1400, 89)
Dimensiones del dataset de validación: (1401, 89)


In [4]:
data = pandas.read_csv('train.csv', sep=';', decimal='.')

<ipython-input-4-b8dd39065073>:1: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pandas.read_csv('train.csv', sep=';', decimal='.')


Eliminacion de filas que no tengan precios, nos ayuda a reducir el numero de imagnes a descargar

In [5]:
data = data.dropna(subset=['Price'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11189 entries, 0 to 11199
Data columns (total 89 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              11189 non-null  int64  
 1   Listing Url                     11189 non-null  object 
 2   Scrape ID                       11189 non-null  int64  
 3   Last Scraped                    11189 non-null  object 
 4   Name                            11189 non-null  object 
 5   Summary                         10741 non-null  object 
 6   Space                           8239 non-null   object 
 7   Description                     11184 non-null  object 
 8   Experiences Offered             11189 non-null  object 
 9   Neighborhood Overview           6909 non-null   object 
 10  Notes                           4221 non-null   object 
 11  Transit                         6859 non-null   object 
 12  Access                          6289 

In [6]:
# Extraer la columna "Precio"
price_column = data['Price']

# Eliminar la columna "Precio" del DataFrame
data.drop('Price', axis=1, inplace=True)

# Insertar la columna "Price" en la primera posición del DataFrame
data.insert(0, 'Price', price_column)

En la columnas como City hay NaN, evaluacion si es un numero pequeno en comparacion con todo el detaset. Si es pequeno se elimina la fila.

In [7]:
valores_unicos = data["City"].unique()
nan_count = data["City"].isna().sum()
# Imprime el conteo de valores NaN en la columna "City"
print("Cantidad de valores NaN en la columna 'City':", nan_count)
# Elimina las filas que contienen NaN en la columna "City"
data = data.dropna(subset=["City"])
# Imprime el DataFrame sin las filas con NaN en "City"
data

Cantidad de valores NaN en la columna 'City': 4


,Price,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
0,22.0,16122949,https://www.airbnb.com/rooms/16122949,20170407214119,2017-04-08,"2£~Casa cueva & Walk My own 2 beds, no bunk-b...",VIVE MADRIZ. Cama y wifi en Bunker -Habitaci...,Camas individuales en apartamento compartido d...,VIVE MADRIZ. Cama y wifi en Bunker -Habitaci...,none,...,9.0,10.0,9.0,NaN,NaN,strict,4,5.04,"40.409548642506394,-3.701254969137256","Host Is Superhost,Host Has Profile Pic,Is Loca..."
1,40.0,15047803,https://www.airbnb.com/rooms/15047803,20170407214119,2017-04-08,Confortable Loft in Olavide (Downtown),Simple and confortable Loft to stay in Madrid ...,This loft is up to cuples or singles who don't...,Simple and confortable Loft to stay in Madrid ...,none,...,10.0,10.0,9.0,NaN,NaN,moderate,3,4.84,"40.43395499885526,-3.700104536563879","Host Has Profile Pic,Host Identity Verified,Re..."
2,32.0,7495485,https://www.airbnb.com/rooms/7495485,20170407214119,2017-04-08,Habitacion para Vegetarianos 2,Habitacion en el centro de Madrid. Cerca de Mu...,"Simpática y silenciosa habitación, en edificio...",Habitacion en el centro de Madrid. Cerca de Mu...,none,...,10.0,10.0,9.0,NaN,NaN,flexible,3,4.24,"40.41641817294734,-3.696901743131365","Host Has Profile Pic,Host Identity Verified,Re..."
3,102.0,76617,https://www.airbnb.com/rooms/76617,20170407214119,2017-04-08,Plaza Comendadoras Lux,NaN,Plaza Comendadoras Lux Apartment: This comfort...,Plaza Comendadoras Lux Apartment: This comfort...,none,...,9.0,9.0,8.0,NaN,NaN,strict,11,0.24,"40.42791316481744,-3.7099928761218366","Host Has Profile Pic,Host Identity Verified,Re..."
4,16.0,13469305,https://www.airbnb.com/rooms/13469305,20170407214119,2017-04-08,Les-friendly room close to city centre,We are three friends and a cat living in this ...,NaN,We are three friends and a cat living in this ...,none,...,10.0,10.0,10.0,NaN,NaN,flexible,1,1.34,"40.40237008242663,-3.696722555307174","Host Is Superhost,Host Has Profile Pic,Require..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11195,19.0,5508219,https://www.airbnb.com/rooms/5508219,20170407214119,2017-04-08,Room 10 min by foot from Madrid Río,"Private room, with a lot of light in a complet...","A quiet place, where you can hear birds tweeti...","Private room, with a lot of light in a complet...",none,...,10.0,9.0,10.0,NaN,NaN,flexible,1,0.17,"40.393040051990575,-3.719456686081845","Host Has Profile Pic,Host Identity Verified,Is..."
11196,59.0,12450824,https://www.airbnb.com/rooms/12450824,20170407214119,2017-04-08,CHARMING APARTMENT (LA LATINA),The apartment is located in one of the most po...,"The apartment , with an elegant design , is lo...",The apartment is located in one of the most po...,none,...,10.0,10.0,10.0,NaN,NaN,moderate,1,6.24,"40.409402726502485,-3.7074466909890362","Host Is Superhost,Host Has Profile Pic,Is Loca..."
11197,175.0,2328467,https://www.airbnb.com/rooms/2328467,20170315084710,2017-03-15,R.116 Nice country house with pool in Pollensa,Country house in the outskirts of Pollensa in ...,Country house in the outskirts of Pollença in ...,Country house in the outskirts of Pollensa in ...,none,...,10.0,10.0,8.0,NaN,NaN,strict,129,0.11,"39.896735915204076,3.027845665676873","Host Has Profile Pic,Host Identity Verified,Is..."
11198,35.0,18047501,https://www.airbnb.com/rooms/18047501,20170407214119,2017-04-08,Fuencarral-Centre Room,"Habitación interior privada con baño, con vent...",NaN,"Habitación interior privada con baño, con vent...",none,...,NaN,NaN,NaN,NaN,NaN,flexible,1,NaN,"40.42276333420992,-3.7005781629888395","Requires License,Instant Bookable"


Para este analisis solo selecionaremos la ciudad con mayor muestras, en este casa: Madrid

In [8]:
# Filtra los valores de la columna "City" que contienen "Madrid" (sin distinguir mayúsculas y minúsculas)
filtro = data["City"].str.contains("Madrid", case=False)

# Aplica el filtro al DataFrame
resultados = data[filtro]

#porcentaje eliminado
print((len(data["City"]) - len(resultados))/len(data["City"])*100)

5.4626732230666075


Para predecir el precio del airbnb, asumimos que no necesitamos informacion relacionada al regitro de los datos, del host y ninguna url. Mantener **Thumbnail Url** necesario para descargar las imagenes.

In [9]:
# Elimina las columnas especificadas del DataFrame
columnas_a_eliminar = ["ID","Listing Url","Scrape ID","Last Scraped","Name","Summary","Space","Description",
                       "Experiences Offered","Neighborhood Overview","Notes","Transit","Access","Interaction",
                       "House Rules","Medium Url","Picture Url","XL Picture Url",
                       "Host ID","Host URL","Host Name","Host Since","Host Location",
                       "Host About", "Host Response Time", "Host Response Rate", "Host Acceptance Rate",
                       "Host Thumbnail Url", "Host Picture Url", "Host Neighbourhood", "Host Listings Count",
                       "Host Total Listings Count", "Host Verifications", "Street", "Neighbourhood",
                       "Neighbourhood Cleansed", "Neighbourhood Group Cleansed", "City","State", "Market", "Smart Location",
                       "Country Code", "Country", "Latitude", "Longitude", "Amenities", "Weekly Price", "Monthly Price", "Calendar Updated",
                       "Has Availability","Availability 30","Availability 60","Availability 90","Availability 365","Calendar last Scraped","First Review",
                       "Last Review","Review Scores Rating","Review Scores Accuracy","Review Scores Cleanliness","Review Scores Checkin","Review Scores Communication",
                       "Review Scores Location","Review Scores Value","License","Jurisdiction Names","Calculated host listings count","Reviews per Month",
                       "Geolocation","Features"]
resultados = resultados.drop(columns=columnas_a_eliminar)

In [10]:
resultados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10574 entries, 0 to 11199
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Price                10574 non-null  float64
 1   Thumbnail Url        8621 non-null   object 
 2   Zipcode              10229 non-null  object 
 3   Property Type        10574 non-null  object 
 4   Room Type            10574 non-null  object 
 5   Accommodates         10574 non-null  int64  
 6   Bathrooms            10537 non-null  float64
 7   Bedrooms             10557 non-null  float64
 8   Beds                 10540 non-null  float64
 9   Bed Type             10574 non-null  object 
 10  Square Feet          421 non-null    float64
 11  Security Deposit     4560 non-null   float64
 12  Cleaning Fee         6263 non-null   float64
 13  Guests Included      10574 non-null  int64  
 14  Extra People         10574 non-null  int64  
 15  Minimum Nights       10574 non-null  int6

In [11]:
#resultados["Zipcode"].fillna("28012", inplace=True)

In [12]:
if "Zipcode" in resultados.columns:
    resultados["Zipcode"] = resultados["Zipcode"].astype(str).str.replace(r'\D', '', regex=True)
    resultados["Zipcode"].replace('', "28012", inplace=True)  # Reemplaza las cadenas vacías con "28012"
resultados["Zipcode"].fillna("28012", inplace=True)  
resultados["Zipcode"] = resultados["Zipcode"].astype(int)


In [13]:
resultados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10574 entries, 0 to 11199
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Price                10574 non-null  float64
 1   Thumbnail Url        8621 non-null   object 
 2   Zipcode              10574 non-null  int64  
 3   Property Type        10574 non-null  object 
 4   Room Type            10574 non-null  object 
 5   Accommodates         10574 non-null  int64  
 6   Bathrooms            10537 non-null  float64
 7   Bedrooms             10557 non-null  float64
 8   Beds                 10540 non-null  float64
 9   Bed Type             10574 non-null  object 
 10  Square Feet          421 non-null    float64
 11  Security Deposit     4560 non-null   float64
 12  Cleaning Fee         6263 non-null   float64
 13  Guests Included      10574 non-null  int64  
 14  Extra People         10574 non-null  int64  
 15  Minimum Nights       10574 non-null  int6

Imputacion de valores para columnas con valores NaN como: ZipCode uso del mas comun

In [14]:
'''# Cuenta la frecuencia de los códigos postales y ordena los resultados
zipcode_counts = resultados["Zipcode"].value_counts().sort_values(ascending=False)
# Obtener el código postal más común
codigo_postal_comun = zipcode_counts.index[0]
# Imprimir el código postal más común
print("Código Postal más común:", codigo_postal_comun)
zipcode_sin_nan = resultados.dropna(subset=["Zipcode"])
print(zipcode_sin_nan)
# Filtrar solo el código postal 28012
zipcode_28012 = zipcode_sin_nan[zipcode_sin_nan["Zipcode"] == "28012"]
# Calcula la media y la dispersión de los precios
media = zipcode_28012["Price"].mean()
dispersion = zipcode_28012["Price"].std()

# Imprime la media y la dispersión
print("Media:", media)
print("Dispersión:", dispersion)

resultados["Zipcode"].fillna("28012", inplace=True)
# Filtrar solo el código postal 28012
zipcode_28012_imputados = resultados[resultados["Zipcode"] == "28012"]

# Calcula la media y la dispersión de los precios
media = zipcode_28012_imputados["Price"].mean()
dispersion = zipcode_28012_imputados["Price"].std()

# Imprime la media y la dispersión
print("Media:", media)
print("Dispersión:", dispersion)
nan_count = resultados["Zipcode"].isna().sum()
print(nan_count)
resultados["Zipcode"] = resultados["Zipcode"].str.replace(r'\D', '', regex=True)
resultados["Zipcode"] = resultados["Zipcode"].astype(float)
resultados.info()'''

'# Cuenta la frecuencia de los códigos postales y ordena los resultados\nzipcode_counts = resultados["Zipcode"].value_counts().sort_values(ascending=False)\n# Obtener el código postal más común\ncodigo_postal_comun = zipcode_counts.index[0]\n# Imprimir el código postal más común\nprint("Código Postal más común:", codigo_postal_comun)\nzipcode_sin_nan = resultados.dropna(subset=["Zipcode"])\nprint(zipcode_sin_nan)\n# Filtrar solo el código postal 28012\nzipcode_28012 = zipcode_sin_nan[zipcode_sin_nan["Zipcode"] == "28012"]\n# Calcula la media y la dispersión de los precios\nmedia = zipcode_28012["Price"].mean()\ndispersion = zipcode_28012["Price"].std()\n\n# Imprime la media y la dispersión\nprint("Media:", media)\nprint("Dispersión:", dispersion)\n\nresultados["Zipcode"].fillna("28012", inplace=True)\n# Filtrar solo el código postal 28012\nzipcode_28012_imputados = resultados[resultados["Zipcode"] == "28012"]\n\n# Calcula la media y la dispersión de los precios\nmedia = zipcode_28012_i

Imputacion de la mediana a valores NAN en Bathrooms, Bedrooms, Beds, Square Feet, Security Deposit, Cleaning Fee

In [15]:
# Imputación con la mediana
median_bathrooms = resultados["Bathrooms"].median()
bathrooms_sin_nan = resultados.dropna(subset=["Bathrooms"])

bathrom_ = bathrooms_sin_nan[bathrooms_sin_nan["Bathrooms"] == median_bathrooms]
print(median_bathrooms)
# Calcula la media y la dispersión de los precios
media = bathrom_["Price"].mean()
dispersion = bathrom_["Price"].std()

# Imprime la media y la dispersión
print("Media bathroom antes:", media)
print("Dispersión bathroom antes:", dispersion)
resultados["Bathrooms"].fillna(median_bathrooms, inplace=True)
bathrom_imputado = resultados[resultados["Bathrooms"] == median_bathrooms]

# Calcula la media y la dispersión de los precios
media = bathrom_imputado["Price"].mean()
dispersion = bathrom_imputado["Price"].std()

# Imprime la media y la dispersión
print("Media bathroom despues:", media)
print("Dispersión bathroom despues:", dispersion)

# Imputación con la mediana
median_Bedrooms = resultados["Bedrooms"].median()
Bedrooms_sin_nan = resultados.dropna(subset=["Bedrooms"])

Bedrooms_ = Bedrooms_sin_nan[Bedrooms_sin_nan["Bedrooms"] == median_Bedrooms]

# Calcula la media y la dispersión de los precios
media = Bedrooms_["Price"].mean()
dispersion = Bedrooms_["Price"].std()

# Imprime la media y la dispersión
print("Media Bedrooms antes:", media)
print("Dispersión Bedrooms antes:", dispersion)
resultados["Bedrooms"].fillna(median_Bedrooms, inplace=True)
Bedrooms_imputado = resultados[resultados["Bedrooms"] == median_Bedrooms]

# Calcula la media y la dispersión de los precios
media = Bedrooms_imputado["Price"].mean()
dispersion = Bedrooms_imputado["Price"].std()

# Imprime la media y la dispersión
print("Media Bedrooms despues:", media)
print("Dispersión Bedrooms despues:", dispersion)

# Imputación con la mediana
median_Beds = resultados["Beds"].median()
Beds_sin_nan = resultados.dropna(subset=["Beds"])

Beds_ = Beds_sin_nan[Beds_sin_nan["Beds"] == median_Beds]

# Calcula la media y la dispersión de los precios
media = Beds_["Price"].mean()
dispersion = Beds_["Price"].std()

# Imprime la media y la dispersión
print("Media Beds antes:", media)
print("Dispersión Beds antes:", dispersion)
resultados["Beds"].fillna(median_Beds, inplace=True)
Beds_imputado = resultados[resultados["Beds"] == median_Beds]

# Calcula la media y la dispersión de los precios
media = Beds_imputado["Price"].mean()
dispersion = Beds_imputado["Price"].std()

# Imprime la media y la dispersión
print("Media Beds despues:", media)
print("Dispersión Beds despues:", dispersion)


1.0
Media bathroom antes: 56.65086848635236
Dispersión bathroom antes: 36.45275052327768
Media bathroom despues: 56.775595899715945
Dispersión bathroom despues: 36.75970412087228
Media Bedrooms antes: 48.625017450788775
Dispersión Bedrooms antes: 34.52452555336335
Media Bedrooms despues: 48.614066852367685
Dispersión Bedrooms despues: 34.51386784038831
Media Beds antes: 66.65916175961205
Dispersión Beds antes: 38.40484733844426
Media Beds despues: 66.6412187606984
Dispersión Beds despues: 38.80231774102879


Imputacion de valores de la mediana de Security Deposit y Cleaning Fee

In [16]:
# Imputación con la mediana
median_SD= resultados["Security Deposit"].median()
SD_sin_nan = resultados.dropna(subset=["Security Deposit"])

SD_ = SD_sin_nan[SD_sin_nan["Security Deposit"] == median_SD]

# Calcula la media y la dispersión de los precios
media = SD_["Price"].mean()
dispersion = SD_["Price"].std()

# Imprime la media y la dispersión
print("Media Security Deposit antes:", media)
print("Dispersión Security Deposit antes:", dispersion)
resultados["Security Deposit"].fillna(median_SD, inplace=True)
SD_imputado = resultados[resultados["Security Deposit"] == median_SD]

# Calcula la media y la dispersión de los precios
media = SD_imputado["Price"].mean()
dispersion = SD_imputado["Price"].std()

# Imprime la media y la dispersión
print("Media Security Deposit despues:", media)
print("Dispersión Security Deposit despues:", dispersion)

# Imputación con la mediana
median_CF= resultados["Cleaning Fee"].median()
CF_sin_nan = resultados.dropna(subset=["Cleaning Fee"])

CF_ = CF_sin_nan[CF_sin_nan["Cleaning Fee"] == median_CF]

# Calcula la media y la dispersión de los precios
media = CF_["Price"].mean()
dispersion = CF_["Price"].std()

# Imprime la media y la dispersión
print("Media Cleaning Fee antes:", media)
print("Dispersión Cleaning Fee antes:", dispersion)
resultados["Cleaning Fee"].fillna(median_CF, inplace=True)

CF_imputado = resultados[resultados["Cleaning Fee"] == median_CF]

# Calcula la media y la dispersión de los precios
media = CF_imputado["Price"].mean()
dispersion = CF_imputado["Price"].std()

# Imprime la media y la dispersión
print("Media Cleaning Fee despues:", media)
print("Dispersión Cleaning Fee despues:", dispersion)

Media Security Deposit antes: 73.2110497237569
Dispersión Security Deposit antes: 31.838028437070236
Media Security Deposit despues: 58.33574215927157
Dispersión Security Deposit despues: 52.53404720815642
Media Cleaning Fee antes: 64.59154929577464
Dispersión Cleaning Fee antes: 23.730751577540804
Media Cleaning Fee despues: 55.44783767165403
Dispersión Cleaning Fee despues: 50.13866887116256


In [17]:
resultados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10574 entries, 0 to 11199
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Price                10574 non-null  float64
 1   Thumbnail Url        8621 non-null   object 
 2   Zipcode              10574 non-null  int64  
 3   Property Type        10574 non-null  object 
 4   Room Type            10574 non-null  object 
 5   Accommodates         10574 non-null  int64  
 6   Bathrooms            10574 non-null  float64
 7   Bedrooms             10574 non-null  float64
 8   Beds                 10574 non-null  float64
 9   Bed Type             10574 non-null  object 
 10  Square Feet          421 non-null    float64
 11  Security Deposit     10574 non-null  float64
 12  Cleaning Fee         10574 non-null  float64
 13  Guests Included      10574 non-null  int64  
 14  Extra People         10574 non-null  int64  
 15  Minimum Nights       10574 non-null  int6

Si imputo por una valor constante como en los casos anteriores estaria rellenando casi el 96 % de los datos de Square Feet. Asi que esta caracteristica no se usara para entrenar el modelo. Varias caracteristicas podrian categorizarse pero no se utilizaran para este analisis

In [18]:
resultados.drop("Square Feet", axis=1, inplace=True)
resultados = resultados.drop(['Property Type', 'Room Type', 'Bed Type', 'Cancellation Policy'], axis=1)
resultados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10574 entries, 0 to 11199
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Price              10574 non-null  float64
 1   Thumbnail Url      8621 non-null   object 
 2   Zipcode            10574 non-null  int64  
 3   Accommodates       10574 non-null  int64  
 4   Bathrooms          10574 non-null  float64
 5   Bedrooms           10574 non-null  float64
 6   Beds               10574 non-null  float64
 7   Security Deposit   10574 non-null  float64
 8   Cleaning Fee       10574 non-null  float64
 9   Guests Included    10574 non-null  int64  
 10  Extra People       10574 non-null  int64  
 11  Minimum Nights     10574 non-null  int64  
 12  Maximum Nights     10574 non-null  int64  
 13  Number of Reviews  10574 non-null  int64  
dtypes: float64(6), int64(7), object(1)
memory usage: 1.2+ MB


In [19]:
# ¿De qué variables disponemos?
resultados.columns

Index(['Price', 'Thumbnail Url', 'Zipcode', 'Accommodates', 'Bathrooms',
       'Bedrooms', 'Beds', 'Security Deposit', 'Cleaning Fee',
       'Guests Included', 'Extra People', 'Minimum Nights', 'Maximum Nights',
       'Number of Reviews'],
      dtype='object')

La columna necesaria "Thumbnail Url" para descargar los datos de imagenes se colocaran en la primera columna

In [20]:
# Obtener el índice de la columna "Thumbnail Url"
cols = list(resultados.columns)
column_index = resultados.columns.get_loc("Thumbnail Url")
cols.append(cols.pop(column_index))
resultados = resultados[cols]


## Descarga de imágenes

In [21]:
def download_images(paths: list,
                    canvas: tuple = (224, 224),
                    nb_channels: int = 3,
                    max_imgs: Optional[int] = None
                    ) -> tuple:
  """ Download a list of images from url adresses, converting them to a specific
  canvas size.

  Args:
    paths: Paths or url adresses from which to load images.
    canvas: Desired image width and height.
    nb_channels: Channels in images (1 for B/W, 3 for RGB).
    max_imgs: Upper threshold in the number of images to download.

  Return:
    a tuple of:
      - image values
      - indices within the paths that were successfull.

  """
  n_images = len(paths) if not max_imgs else max_imgs
  images = numpy.zeros((n_images, canvas[0], canvas[1], nb_channels),
                       dtype=numpy.uint8)
  downloaded_idxs = []

  for i_img, url in enumerate(tqdm(paths, total=n_images)):
    if i_img >= n_images:
      break
    try:
        img = io.imread(url)
        img = cv2.resize(img, (canvas[0], canvas[1]))
        downloaded_idxs.append(i_img)
        images[i_img] = img
    except (IOError, ValueError) as e:  # Unavailable url / conversion error
        pass
  return images[downloaded_idxs], downloaded_idxs

# Train

Normalización y conversión en el rango [0, 1]

In [22]:
imagesTrain, idxs_Train = download_images(resultados['Thumbnail Url'], max_imgs=None)
imagesTrain = imagesTrain.astype("float32") / 255.
imagesTrain.shape

100%|██████████| 10574/10574 [50:28<00:00,  3.49it/s]  


(6762, 224, 224, 3)

In [23]:
filtered_data_Train = resultados.iloc[idxs_Train]
filtered_data_Train

,Price,Zipcode,Accommodates,Bathrooms,Bedrooms,Beds,Security Deposit,Cleaning Fee,Guests Included,Extra People,Minimum Nights,Maximum Nights,Number of Reviews,Thumbnail Url
0,22.0,28012,2,1.0,1.0,2.0,150.0,5.0,1,0,1,4,20,https://a0.muscache.com/im/pictures/63f14611-6...
1,40.0,28010,2,1.0,0.0,1.0,150.0,10.0,1,0,2,1125,30,https://a0.muscache.com/im/pictures/90af47ae-0...
2,32.0,28014,3,1.0,1.0,3.0,150.0,25.0,1,11,1,1125,82,https://a0.muscache.com/im/pictures/95683644/1...
3,102.0,28015,5,1.0,1.0,3.0,300.0,42.0,1,11,3,365,16,https://a0.muscache.com/im/pictures/514847/938...
6,45.0,28014,4,1.0,1.0,2.0,150.0,20.0,2,15,2,1125,131,https://a0.muscache.com/im/pictures/76405548/a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11193,21.0,280050,2,1.0,1.0,2.0,150.0,10.0,1,15,1,1125,0,https://a0.muscache.com/im/pictures/f50d4d7b-7...
11195,19.0,280190,1,1.0,1.0,1.0,150.0,25.0,1,0,1,1125,4,https://a0.muscache.com/im/pictures/68735078/5...
11196,59.0,280050,2,1.0,1.0,1.0,150.0,10.0,2,0,2,1125,74,https://a0.muscache.com/im/pictures/5527c6ce-f...
11198,35.0,28012,2,1.0,1.0,2.0,150.0,25.0,1,0,2,20,0,https://a0.muscache.com/im/pictures/103d4c11-0...


In [24]:
numpy.save('imageTrain.npy', imagesTrain)
filtered_data_Train.to_csv('filtered_data_Train.csv', sep=';', index=False)

In [25]:
# Comprobamos a abrirlos de nuevo 
saved_data_Train = pandas.read_csv("filtered_data_Train.csv", sep=';')
saved_imgs_Train = numpy.load("imageTrain.npy")
saved_data_Train.shape, saved_imgs_Train.shape

((6762, 14), (6762, 224, 224, 3))

In [26]:
y_train = saved_data_Train.iloc[:, 0:1]   # Seleccionamos la primera columna, price
X_train = saved_data_Train.iloc[:, 1:-1]  # Seleccionamos todas las columnas excepto la primera y la última

scaler = preprocessing.StandardScaler().fit(X_train)
XtrainScaled = scaler.transform(X_train)

X_train = XtrainScaled 

# Test

In [27]:
dataTest = pandas.read_csv('test.csv', sep=';', decimal='.')
# Extraer la columna "Precio"
price_column = dataTest['Price']
# Eliminar la columna "Precio" del dataTestTestFrame
dataTest.drop('Price', axis=1, inplace=True)
# Insertar la columna "Price" en la primera posición del dataTestTestFrame
dataTest.insert(0, 'Price', price_column)

dataTest = dataTest.dropna(subset=["City"])
# Filtra los valores de la columna "City" que contienen "Madrid" (sin distinguir mayúsculas y minúsculas)
filtro = dataTest["City"].str.contains("Madrid", case=False)

# Aplica el filtro al dataTestTestTestFrame
resultadosTest = dataTest[filtro]
# Elimina las columnas especificadas del dataTestTestTestFrame
columnas_a_eliminar = ["ID","Listing Url","Scrape ID","Last Scraped","Name","Summary","Space","Description",
                       "Experiences Offered","Neighborhood Overview","Notes","Transit","Access","Interaction",
                       "House Rules","Medium Url","Picture Url","XL Picture Url",
                       "Host ID","Host URL","Host Name","Host Since","Host Location",
                       "Host About", "Host Response Time", "Host Response Rate", "Host Acceptance Rate",
                       "Host Thumbnail Url", "Host Picture Url", "Host Neighbourhood", "Host Listings Count",
                       "Host Total Listings Count", "Host Verifications", "Street", "Neighbourhood",
                       "Neighbourhood Cleansed", "Neighbourhood Group Cleansed", "City","State", "Market", "Smart Location",
                       "Country Code", "Country", "Latitude", "Longitude", "Amenities", "Weekly Price", "Monthly Price", "Calendar Updated",
                       "Has Availability","Availability 30","Availability 60","Availability 90","Availability 365","Calendar last Scraped","First Review",
                       "Last Review","Review Scores Rating","Review Scores Accuracy","Review Scores Cleanliness","Review Scores Checkin","Review Scores Communication",
                       "Review Scores Location","Review Scores Value","License","Jurisdiction Names","Calculated host listings count","Reviews per Month",
                       "Geolocation","Features"]
resultadosTest = resultadosTest.drop(columns=columnas_a_eliminar)

resultadosTest = resultadosTest[resultadosTest["Price"].notna()]
if "Zipcode" in resultadosTest.columns:
    resultadosTest["Zipcode"] = resultadosTest["Zipcode"].astype(str).str.replace(r'\D', '', regex=True)
    resultadosTest["Zipcode"].replace('', "28012", inplace=True)  # Reemplaza las cadenas vacías con "28012"
resultadosTest["Zipcode"].fillna("28012", inplace=True)  # Rellena los NaN con un valor predeterminado, como "0"
resultadosTest["Zipcode"] = resultadosTest["Zipcode"].astype(int)
resultadosTest["Bathrooms"].fillna(median_bathrooms, inplace=True)
resultadosTest["Bedrooms"].fillna(median_Bedrooms, inplace=True)

resultadosTest["Beds"].fillna(median_Beds, inplace=True)
resultadosTest["Security Deposit"].fillna(median_SD, inplace=True)
resultadosTest["Cleaning Fee"].fillna(median_CF, inplace=True)
resultadosTest.drop("Square Feet", axis=1, inplace=True)
resultadosTest = resultadosTest.drop(['Property Type', 'Room Type', 'Bed Type', 'Cancellation Policy'], axis=1)
colstest = list(resultadosTest.columns)
column_index_test = resultadosTest.columns.get_loc("Thumbnail Url")
colstest.append(colstest.pop(column_index_test))
resultadosTest = resultadosTest[colstest]

Normalización y conversión en el rango [0, 1]

In [28]:
imagesTest, idxs_Test = download_images(resultadosTest['Thumbnail Url'], max_imgs=None)
imagesTest = imagesTest.astype("float32") / 255.
imagesTest.shape

100%|██████████| 1341/1341 [07:25<00:00,  3.01it/s]


(852, 224, 224, 3)

In [29]:
filtered_data_Test = resultadosTest.iloc[idxs_Test]
filtered_data_Test

,Price,Zipcode,Accommodates,Bathrooms,Bedrooms,Beds,Security Deposit,Cleaning Fee,Guests Included,Extra People,Minimum Nights,Maximum Nights,Number of Reviews,Thumbnail Url
1,22.0,280120,8,1.0,1.0,8.0,150.0,25.0,1,22,1,1125,15,https://a0.muscache.com/im/pictures/47542233/c...
3,149.0,28012,2,1.0,1.0,1.0,150.0,77.0,2,0,3,1125,1,https://a0.muscache.com/im/pictures/c7d30f39-7...
5,109.0,280040,4,2.0,1.0,2.0,150.0,30.0,2,15,2,1125,6,https://a0.muscache.com/im/pictures/c8c81bb9-9...
6,40.0,280050,2,1.5,1.0,2.0,100.0,25.0,1,15,5,1125,0,https://a0.muscache.com/im/pictures/dc31bb3b-f...
8,35.0,280150,2,1.0,0.0,2.0,92.0,25.0,1,10,7,1125,3,https://a0.muscache.com/im/pictures/d4bb05a2-e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,41.0,280080,2,1.0,1.0,2.0,150.0,25.0,1,9,2,1125,9,https://a0.muscache.com/im/pictures/57081610/f...
1395,27.0,280140,2,1.0,1.0,1.0,200.0,10.0,1,10,2,365,110,https://a0.muscache.com/im/pictures/13595089/e...
1396,62.0,280050,4,1.0,1.0,2.0,150.0,20.0,2,15,1,1125,105,https://a0.muscache.com/im/pictures/eee1b1c8-6...
1398,70.0,280050,5,1.0,1.0,3.0,200.0,25.0,4,10,2,1125,2,https://a0.muscache.com/im/pictures/2d80dc99-b...


In [30]:
numpy.save('imagesTest.npy', imagesTest)
filtered_data_Test.to_csv('filtered_data_Test.csv', sep=';', index=False)

In [31]:
# Comprobamos a abrirlos de nuevo
saved_data_Test = pandas.read_csv("filtered_data_Test.csv", sep=';')
saved_imgs_Test = numpy.load("imagesTest.npy")
saved_data_Test.shape, saved_imgs_Test.shape

((852, 14), (852, 224, 224, 3))

In [32]:
y_test = saved_data_Test.iloc[:, 0:1]      # nos quedamos con la 1ª columna, price
X_test = saved_data_Test.iloc[:,1:-1]      # nos quedamos con el resto


# recordad que esta normalización/escalado la realizo con el scaler anterior, basado en los datos de training!
XtestScaled = scaler.transform(X_test)
X_test = XtestScaled

# Validacion

In [33]:
dataVal = pandas.read_csv('validation.csv', sep=';', decimal='.')
# Extraer la columna "Precio"
price_column = dataVal['Price']
# Eliminar la columna "Precio" del dataVal
dataVal.drop('Price', axis=1, inplace=True)
# Insertar la columna "Price" en la primera posición del dataVal
dataVal.insert(0, 'Price', price_column)

dataVal = dataVal.dropna(subset=["City"])
# Filtra los valores de la columna "City" que contienen "Madrid" (sin distinguir mayúsculas y minúsculas)
filtro = dataVal["City"].str.contains("Madrid", case=False)

# Aplica el filtro al dataVal
resultadosVal = dataVal[filtro]
# Elimina las columnas especificadas del dataVal
columnas_a_eliminar = ["ID","Listing Url","Scrape ID","Last Scraped","Name","Summary","Space","Description",
                       "Experiences Offered","Neighborhood Overview","Notes","Transit","Access","Interaction",
                       "House Rules","Medium Url","Picture Url","XL Picture Url",
                       "Host ID","Host URL","Host Name","Host Since","Host Location",
                       "Host About", "Host Response Time", "Host Response Rate", "Host Acceptance Rate",
                       "Host Thumbnail Url", "Host Picture Url", "Host Neighbourhood", "Host Listings Count",
                       "Host Total Listings Count", "Host Verifications", "Street", "Neighbourhood",
                       "Neighbourhood Cleansed", "Neighbourhood Group Cleansed", "City","State", "Market", "Smart Location",
                       "Country Code", "Country", "Latitude", "Longitude", "Amenities", "Weekly Price", "Monthly Price", "Calendar Updated",
                       "Has Availability","Availability 30","Availability 60","Availability 90","Availability 365","Calendar last Scraped","First Review",
                       "Last Review","Review Scores Rating","Review Scores Accuracy","Review Scores Cleanliness","Review Scores Checkin","Review Scores Communication",
                       "Review Scores Location","Review Scores Value","License","Jurisdiction Names","Calculated host listings count","Reviews per Month",
                       "Geolocation","Features"]
resultadosVal = resultadosVal.drop(columns=columnas_a_eliminar)

resultadosVal = resultadosVal[resultadosVal["Price"].notna()]

if "Zipcode" in resultadosVal.columns:
    resultadosVal["Zipcode"] = resultadosVal["Zipcode"].astype(str).str.replace(r'\D', '', regex=True)
    resultadosVal["Zipcode"].replace('', "28012", inplace=True)  # Reemplaza las cadenas vacías con "28012"
resultadosVal["Zipcode"].fillna("28012", inplace=True)  # Rellena los NaN con un valor predeterminado, como "0"
resultadosVal["Zipcode"] = resultadosVal["Zipcode"].astype(int)
resultadosVal["Bathrooms"].fillna(median_bathrooms, inplace=True)
resultadosVal["Bedrooms"].fillna(median_Bedrooms, inplace=True)

resultadosVal["Beds"].fillna(median_Beds, inplace=True)
resultadosVal["Security Deposit"].fillna(median_SD, inplace=True)
resultadosVal["Cleaning Fee"].fillna(median_CF, inplace=True)
resultadosVal.drop("Square Feet", axis=1, inplace=True)
resultadosVal = resultadosVal.drop(['Property Type', 'Room Type', 'Bed Type', 'Cancellation Policy'], axis=1)

colsval = list(resultadosVal.columns)
print(colsval)
column_index_val =  resultadosVal.columns.get_loc("Thumbnail Url")
colsval.append(colsval.pop(column_index_val))
resultadosVal = resultadosVal[colsval]


['Price', 'Thumbnail Url', 'Zipcode', 'Accommodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Security Deposit', 'Cleaning Fee', 'Guests Included', 'Extra People', 'Minimum Nights', 'Maximum Nights', 'Number of Reviews']


Normalización y conversión en el rango [0, 1]

In [34]:
imagesVal, idxs_Val = download_images(resultadosVal['Thumbnail Url'], max_imgs=None)
imagesVal = imagesVal.astype("float32") / 255.
imagesVal.shape

100%|██████████| 1321/1321 [07:53<00:00,  2.79it/s]


(839, 224, 224, 3)

In [35]:
filtered_data_Val = resultadosVal.iloc[idxs_Val]
filtered_data_Val

,Price,Zipcode,Accommodates,Bathrooms,Bedrooms,Beds,Security Deposit,Cleaning Fee,Guests Included,Extra People,Minimum Nights,Maximum Nights,Number of Reviews,Thumbnail Url
1,30.0,280130,2,1.0,1.0,1.0,150.0,25.0,1,15,1,30,250,https://a0.muscache.com/im/pictures/6e38aabe-9...
3,50.0,280040,2,1.0,1.0,1.0,150.0,25.0,1,0,2,10,22,https://a0.muscache.com/im/pictures/107390143/...
5,95.0,280020,2,1.0,2.0,1.0,200.0,40.0,1,0,4,1125,0,https://a0.muscache.com/im/pictures/815ee1e7-e...
6,536.0,280140,7,4.0,4.0,5.0,150.0,103.0,1,0,3,1125,0,https://a0.muscache.com/im/pictures/38b074c3-1...
7,16.0,280190,2,1.0,1.0,1.0,150.0,5.0,1,10,3,1125,2,https://a0.muscache.com/im/pictures/f9cfb13e-4...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1392,120.0,280040,4,1.0,1.0,2.0,100.0,25.0,2,30,2,1125,2,https://a0.muscache.com/im/pictures/aa83c8d8-5...
1393,50.0,280080,2,1.5,1.0,2.0,150.0,25.0,1,0,1,1125,7,https://a0.muscache.com/im/pictures/7d31cf89-8...
1396,55.0,280060,2,1.0,1.0,2.0,150.0,25.0,1,0,2,1125,2,https://a0.muscache.com/im/pictures/d3dd82f1-3...
1397,40.0,280260,2,1.0,1.0,2.0,150.0,5.0,1,0,1,1125,3,https://a0.muscache.com/im/pictures/86653357/f...


In [36]:
numpy.save('imagesVal.npy', imagesVal)
filtered_data_Val.to_csv('filtered_data_Val.csv', sep=';', index=False)

In [37]:
# Comprobamos a abrirlos de nuevo
saved_data_Val = pandas.read_csv("filtered_data_Val.csv", sep=';')
saved_imgs_Val = numpy.load("imagesVal.npy")
saved_data_Val.shape, saved_imgs_Val.shape

((839, 14), (839, 224, 224, 3))

In [38]:
y_val = saved_data_Val.iloc[:,0:1]     # nos quedamos con la 1ª columna, price
X_val = saved_data_Val.iloc[:,1:-1]      # nos quedamos con el resto


# recordad que esta normalización/escalado la realizo con el scaler anterior, basado en los datos de training!
XvalScaled = scaler.transform(X_val)

X_test = XvalScaled

Así, por ejemplo, cargamos y visualizamos la distribución de las etiquetas:

In [39]:
saved_data_Train['Price'].hist(bins=100)
saved_data_Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6762 entries, 0 to 6761
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Price              6762 non-null   float64
 1   Zipcode            6762 non-null   int64  
 2   Accommodates       6762 non-null   int64  
 3   Bathrooms          6762 non-null   float64
 4   Bedrooms           6762 non-null   float64
 5   Beds               6762 non-null   float64
 6   Security Deposit   6762 non-null   float64
 7   Cleaning Fee       6762 non-null   float64
 8   Guests Included    6762 non-null   int64  
 9   Extra People       6762 non-null   int64  
 10  Minimum Nights     6762 non-null   int64  
 11  Maximum Nights     6762 non-null   int64  
 12  Number of Reviews  6762 non-null   int64  
 13  Thumbnail Url      6762 non-null   object 
dtypes: float64(6), int64(7), object(1)
memory usage: 739.7+ KB


In [40]:
saved_data_Test['Price'].hist(bins=100)
saved_data_Test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Price              852 non-null    float64
 1   Zipcode            852 non-null    int64  
 2   Accommodates       852 non-null    int64  
 3   Bathrooms          852 non-null    float64
 4   Bedrooms           852 non-null    float64
 5   Beds               852 non-null    float64
 6   Security Deposit   852 non-null    float64
 7   Cleaning Fee       852 non-null    float64
 8   Guests Included    852 non-null    int64  
 9   Extra People       852 non-null    int64  
 10  Minimum Nights     852 non-null    int64  
 11  Maximum Nights     852 non-null    int64  
 12  Number of Reviews  852 non-null    int64  
 13  Thumbnail Url      852 non-null    object 
dtypes: float64(6), int64(7), object(1)
memory usage: 93.3+ KB


In [41]:
saved_data_Val['Price'].hist(bins=100)
saved_data_Val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839 entries, 0 to 838
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Price              839 non-null    float64
 1   Zipcode            839 non-null    int64  
 2   Accommodates       839 non-null    int64  
 3   Bathrooms          839 non-null    float64
 4   Bedrooms           839 non-null    float64
 5   Beds               839 non-null    float64
 6   Security Deposit   839 non-null    float64
 7   Cleaning Fee       839 non-null    float64
 8   Guests Included    839 non-null    int64  
 9   Extra People       839 non-null    int64  
 10  Minimum Nights     839 non-null    int64  
 11  Maximum Nights     839 non-null    int64  
 12  Number of Reviews  839 non-null    int64  
 13  Thumbnail Url      839 non-null    object 
dtypes: float64(6), int64(7), object(1)
memory usage: 91.9+ KB


In [42]:
# Verificar si hay valores NaN en alguna columna del DataFrame
if saved_data_Test.isna().any().any():
    print("Hay valores NaN en el DataFrame saved_data_Test")
else:
    print("No hay valores NaN en el DataFrame saved_data_Test")
# Verificar si hay valores NaN en alguna columna del DataFrame
if saved_data_Train.isna().any().any():
    print("Hay valores NaN en el DataFrame saved_data_Train")
else:
    print("No hay valores NaN en el DataFrame saved_data_Train")
# Verificar si hay valores NaN en alguna columna del DataFrame
if saved_data_Val.isna().any().any():
    print("Hay valores NaN en el DataFrame saved_data_Val")
else:
    print("No hay valores NaN en el DataFrame saved_data_Val")


No hay valores NaN en el DataFrame saved_data_Test
No hay valores NaN en el DataFrame saved_data_Train
No hay valores NaN en el DataFrame saved_data_Val
